# Лабораторна Робота 1
### З дисципліни "Методи глибинного навчання"
##### Виконала Неділько Дарина, група КМ-91
**Тема:** «Розробка  програмного  забезпечення  для  реалізації  ймовірнісної
нейронної мережі PNN»

**Завдання:** розробити програмне забезпечення для реалізації мережі PNN. Побудувати мережу PNN, призначену для класифікації об’єктів на А та В.

**Теоретичні відомості**
Мережі PNN є одним з типів ймовірнісних НМ. Зазвичай вони використовуються для задач класифікації. У задачах класифікації виходи інтерпретуються як ймовірності того, що образ належить деякому класу. Під час розв'язання цього типу задачі розрахунок щільності ймовірності відбувається за допомогою методу ядерних оцінок. Ідея цього методу полягає у тому, що якщо спостереження ведеться у певній точці простору ознак класів, то це означає, що в у цій точці є деяка ненульова щільність імовірності. Чим ближче до точки - тим більша щільність. сумарну функціональну оцінку щільності ймовірності можна розрахувати як суму вказанх функцій.
Оцінка функції щільності ймовірності виставляється на основі навчальних образів з використанням методу Парцена. При цьому застосовується вагова функція(ядро), що має центр у точці, яка являє собою навчальний образ. Найчастіше в якості ядра використовують функцію Гауса. Мережа складається із чотирьох шарів нейронів, кількість яких визначається структурою навчальних даних. Кількість вхідних нейронів дорівнює кількості ознак класу. Кількість елементів ШО дорівнює кількості навчальних образів. Вхідний шар і ШО становлять повнозв'язну структуру. Кількість елементів шару додавання(ШД) дорівнює кількості класів. Елемент ШО зв'язаний тільки з тим елементом ШД, якому відповідає клас образу.
Спробуємо розібратися на прикладі завдань для другої лабораторної роботи. Для навчання мережі нам подано чотири образи:

| x1 | x2 | y |
| :- | -: | :-: |
| 1 | 1 | A |
| 1 | 1 | A |
| 2 | 1 | A |
| 10 | 10 | B |


Для цього випаду у вхідному шарі у нас буде 2 нейрони, адже вхідни параметри у нас 2: х1 та х2, у шарі образів у нас буде 4 нейрони, оскільки навчальних образів у нас 4. В шарі додавання у буде 2 нейрони, оскільки у нас лше два класи А та В.
Активність j-того нейрона ШО $\theta_j^o$ розраховується так: $$\theta_j^o = \sum_{i=1}^N{exp(\frac{-(w_{i,j}-x{i})^2}{\sigma^2})},$$
де x - невідомий образ, хі - іта компонента невідомого образу, N - кількість компонент вхідного вектору-образу, $\sigma$ - радіус функції Гауса.
Для зв'язків, що входять в елемент ШО, вагові коефіціенти встановлюютьяс такими ж, як складові частини відповідного навчального прикладу. Тобто усі параметри мережі PNN безпосередньо визначаються навчальними прикладами. Вагові коефіціенти зв'язків, що входять в ШД та вихідний еоемент, дорівнюють 1. Нейронам ШД характерна лінійна функція активацї. Активність j-того нейрону ШД $\theta_j^S$ становить: $$\theta_j^S = \frac{\sum_{i=1}^N{\theta_j^o}}{N},$$де N - кількість нейронів ШО, що пов'язані з j-м нейроном ШД, $\theta_j^o$ - активність i-го нейрону ШО, що пов'язаний з j-тим нейроном ШД.
Значення активності нейрона ШД визначає ймовірність віднесення вхідного образу до класу, що відповідає цьому нейрону. Вихідний елемент визначає тільки нейрон з максимальною активністю. Для процесу навчання мережі PNN важливою є наявність тільки одного керувального параметру навчання - $\sigma$ - радіусу функції Гауса. PNN малочутливі до величини цього радіусу, тому я візьму сигма = 1 .

In [18]:
import numpy as np
import pandas as pd
import math


# пишемо модель
def PNN(df,clasPoint):
    # визначаємо класи
    Classes = df.groupby('y')
    # визначаємо кількість класів
    ClassPat = len(Classes)
    # задаємо словник, щоб зберігати у ньому пораховані у ШД суми
    Sums = dict()
    # задаємо кількість нейронів у вхідному шарі
    EN = df.shape[1]-1

    sigma = 0.5

#     масив хів
    Xs = df.drop(['y'],axis=1).values

    img=0
    for i in Classes.groups.keys():
        Sums[i] = 0
        #   визначаємо кількість нейронів ШО. що відповідають нейронам в ШД для iтого класу
        ClassPoins = len(Classes.get_group(i))
        theta_s = 0

        for j in range(0,ClassPoins):
            # визначаємо значення функції активації для шару ШО
            theta_o = math.exp(-((Xs[img][0] - clasPoint[0])**2+(Xs[img][1] - clasPoint[1])**2)/(sigma**2))

#             визначаємо значення функції активації для шару ШС з кожною ітерацією
            theta_s += theta_o
        Sums[i] = theta_s/ClassPoins
        img+=1

    # дізаємося для якого класу більша ймовірність співпадіння
    res = max(Sums,key = Sums.get)

    print(f'Для образу x={clasPoint} визначено клас {res}')


# задаємо навчальні образи
train = {
    'x1' : [1,1,2,10],
    'x2' : [1,2,1,10],
    'y' : ['A','A','A','B']
}

train_df = pd.DataFrame(data=train)

# розпізнавання
t = np.array([[1,0],[3,4],[12,10]])

for el in t:
    PNN(train_df,el)


Для образу x=[1 0] визначено клас A
Для образу x=[3 4] визначено клас B
Для образу x=[12 10] визначено клас B


**Висновки:** У ході лабораторної роботи було досліджено будову мережі PNN, визначено нейрони у кожному шарі мережі для нашого випадку та натреновано.